# 🌌 P(k) Plot Simulation : N-body

#### Pragun Nepal  
#### June 26, 2025

---

This notebook contains the code for running the P(k) Plot Simulation from Redshift Values 100 to 1. The full video of the simulations have been uploaded in my YouTube channel - https://www.youtube.com/@PragunNepal

All the data for this simulation have been obatined from the N-body Repository by Dr. Rajesh Mondal (GitHub : rajeshmondal18 - https://github.com/rajeshmondal18 )

---

## 📚 Table of Contents

1. [Setting up ffmpeg for Jupyter Lab](#Setting_up_ffmpeg_for_Jupyter_Lab)
2. [Code for P(k) Plot Simulation](#Code_for_N-body_Simulation)
3. [Conclusion](#conclusion)

---

## 🔭 <a name="Setting_up_ffmpeg_for_Jupyter_Lab"></a>1. Setting up ffmpeg for Jupyter Lab



```python
import os
import matplotlib as mpl

#Enter the Full Fath of the ffmeg library - It may be different for you
ffmpeg_path = "/opt/homebrew/bin/ffmpeg"

#Prepend its directory to PATH for this session
os.environ["PATH"] = os.path.dirname(ffmpeg_path) + os.pathsep + os.environ["PATH"]

#Tell Matplotlib exactly where to find it
mpl.rcParams["animation.ffmpeg_path"] = ffmpeg_path

#Verification
import shutil
print("→ PATH is now:", os.environ["PATH"].split(os.pathsep)[:3], "…")
print("→ which(ffmpeg):", shutil.which("ffmpeg"))
print("→ rcParams ffmpeg_path:", mpl.rcParams["animation.ffmpeg_path"])
```



If ffmpeg is not installed in your PC. You can do so by visiting the website - https://ffmpeg.org/download.html



---

## ⚙️ <a name="Code_for_N-body_Simulation"></a>2. Code for P(k) Plot Simulation

```python
import numpy as np
import glob
import re
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, FFMpegWriter

#Parameters
interp_steps = 5         #number of interpolated frames between each original redshift
zoom_x_fraction = 0.2    #fraction of the x-range to display
num_k_ticks = 5          #number of k-axis tick references
y_margin_factor = 1.50
line_width = 4           #line width for data line
spine_width = 2.5        #axis spine width
tick_width = 2.5         #tick width
tick_length = 8          #tick mark length
label_fontsize = 18      #fontsize for axis labels
tick_labelsize = 16      #fontsize for tick labels

#Gathering and sorting files by descending redshift values
files = glob.glob('pk.nbody*.txt')

def extract_redshift(fname):
    m = re.search(r'pk\.nbody([\d\.]+)\.txt$', fname)
    return float(m.group(1)) if m else None

files_sorted = sorted(files, key=lambda f: extract_redshift(f), reverse=True)
redshifts = [extract_redshift(f) for f in files_sorted]

data_list = [np.loadtxt(f) for f in files_sorted]
ks_list = [d[:,0] for d in data_list]
ps_list = [d[:,1] for d in data_list]

#Computing fixed zoom limits across entire dataset
all_x = np.hstack(ks_list)
all_y = np.hstack(ps_list)
#full data limits
x_min, x_max = all_x.min(), all_x.max()
y_min, y_max_orig = all_y.min(), all_y.max()
#apply y-axis margin
y_max = y_max_orig * y_margin_factor
#zoomed x-axis limits (first fraction of range)
x_range = x_max - x_min
x_zoom_max = x_min + zoom_x_fraction * x_range
#compute k-axis tick positions on log scale
k_ticks = np.logspace(np.log10(x_min), np.log10(x_zoom_max), num=num_k_ticks)

#Building interpolated frame series
interp_ks = []
interp_ps = []
interp_zs = []
for i in range(len(ps_list)-1):
    k1, k2 = ks_list[i], ks_list[i+1]
    p1, p2 = ps_list[i], ps_list[i+1]
    for t in range(interp_steps):
        alpha = t / interp_steps
        interp_ks.append(k1)
        interp_ps.append((1-alpha)*p1 + alpha*p2)
        interp_zs.append((1-alpha)*redshifts[i] + alpha*redshifts[i+1])
# append final original frame
interp_ks.append(ks_list[-1])
interp_ps.append(ps_list[-1])
interp_zs.append(redshifts[-1])

#Prepare figure and line object with styling
# 32:13 figure aspect ratio
fig, ax = plt.subplots(figsize=(32, 13), facecolor='black')
ax.set_facecolor('black')

#initial line (golden), thicker
line, = ax.plot([], [], lw=line_width, color='gold')
#title text
title = ax.text(0.5, 1.03, 'N-body Power Spectrum Graph', transform=ax.transAxes, ha='center', fontsize=20, color='purple', fontweight='bold')

#grid styling (both axes)
ax.grid(True, color='purple', linewidth=0.5, linestyle='-')
ax.xaxis.grid(True, which='major', color='purple', linewidth=0.5)
ax.yaxis.grid(True, which='major', color='purple', linewidth=0.5)

#spine (axes) styling
for spine in ax.spines.values():
    spine.set_edgecolor('purple')
    spine.set_linewidth(spine_width)
#tick styling
ax.tick_params(colors='purple', width=tick_width, length=tick_length)
#tick label size and weight
ax.tick_params(labelsize=tick_labelsize)
for label in ax.get_xticklabels() + ax.get_yticklabels():
    label.set_fontweight('bold')

#label colors, size, and weight
ax.xaxis.label.set_color('purple')
ax.yaxis.label.set_color('purple')
ax.xaxis.label.set_size(label_fontsize)
ax.yaxis.label.set_size(label_fontsize)
ax.xaxis.label.set_fontweight('bold')
ax.yaxis.label.set_fontweight('bold')

ax.set_xlabel('k [Mpc$^{-1}$]')
ax.set_ylabel('P(k) [Mpc$^3$]')

#set log-log scaling
ax.set_xscale('log')
ax.set_yscale('log')

#set k-axis ticks for reference
ax.set_xticks(k_ticks)
#enable minor ticks for context
ax.minorticks_on()
#format major tick labels
ax.get_xaxis().set_major_formatter(plt.FuncFormatter(lambda val, pos: f"{val:.2e}"))

#ensure y-axis ticks on both sides for reference
ax.yaxis.set_ticks_position('both')
ax.yaxis.set_label_position('left')

#set fixed zoomed x-axis and adjusted y-axis
ax.set_xlim(x_min, x_zoom_max)
ax.set_ylim(y_min, y_max)

#Setting Animation Functions
def init():
    line.set_data([], [])
    title.set_text('')
    return line, title


def animate(i):
    x = interp_ks[i]
    y = interp_ps[i]
    z = interp_zs[i]
    line.set_data(x, y)
    title.set_text(f'Redshift = {z:.3f}')
    return line, title

#Creating and Saving the Animation
anim = FuncAnimation(
    fig,
    animate,
    init_func=init,
    frames=len(interp_ps),
    interval=100,  # ms per frame
    blit=True
)

#To save as MP4 (ffmpeg required):
writer = FFMpegWriter(fps=interp_steps*5, bitrate=2000)
anim.save('pkfinaltry.mp4', writer=writer)

plt.show()


```
---


## 🔭 <a name="conclusion"></a>3. Conclusion

The code has been made general, hence its easily customisable. Please make necessary changes to the code as per your needs. 

©Pragun Nepal, 2025

Contact Me :
- Email : pragunnepal@gmail.com
- Email : pragun23@iisertvm.ac.in
- GitHub : PragunNepal
- LinkedIn : Pragun Nepal